Qonscious combines backend adapters, figures of merits, merit compliance checks, and actions to abstract what it means to perform resource aware conditional quantum computation in the NISQ era.
Correcto
## Adapters
A BackendAdapter is the abstraction layer in Qonscious that standardizes access to quantum backends, whether they are real devices, cloud simulators, or local emulators. It exposes a uniform interface for executing quantum circuits and retrieving results in a consistent format, including counts, backend characteristics, and execution metadata. This allows Qonscious to remain agnostic to the specifics of backend APIs (such as IBM Sampler or Aer), while still supporting resource introspection, circuit execution, measurement, and runtime resource checks. Developers are free to implement custom adapters to integrate other platforms.

You can create an adapter on Qiskit's SamplerV2 as follows:

In [ ]:
from qonscious.adapters import AerSamplerAdapter
adapter = AerSamplerAdapter()



You can create an adapter to work with one of IBM's fake (simulated computers) as follows:

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qonscious.adapters import IBMSamplerAdapter
adapter = IBMSamplerAdapter(FakeManilaV2()) # edit as needed

In [ ]:
from qonscious.adapters import IonQBackendAdapter
api_key = "zX6nxTjXeClRj930SzRVRV3IrP6yg2NQ"  # Tu API key directamente
adapter = IonQBackendAdapter.aria_simulator(api_key)

# Extension from api to test some things:
As seen; is a test for checking the Qbits of a specified backend, then make a decision based in the decision function "requires_qbits_over(n)" n is equal to an user input(integer)

In [ ]:
import json #import json module for pretty-printing nothing else
def on_pass(backend_adapter, figureOfMeritResults):
    """
    Se ejecuta si la comprobación de mérito es exitosa.
    Imprime las propiedades y el número de qubits.
    """
    firstFoMResult = figureOfMeritResults[0]
    print("\n--- Checks PASSED ---")

    # Accede al diccionario de propiedades de forma segura
    properties = firstFoMResult.get('properties', {})

    # 1. Imprime el diccionario 'properties' completo
    print("\n--- Calculated Properties ---")
    pretty_properties = json.dumps(properties, indent=2)
    print(pretty_properties)

    # 2. Imprime solo el valor de 'n_qubits'
    num_qubits = properties.get('n_qubits', 'N/A')
    print(f"\nNumber of Qubits: {num_qubits}")
    print("-" * 25)

    return None

def on_fail(backend_adapter, figureOfMeritResults):
    """
    Se ejecuta si la comprobación de mérito falla.
    Imprime las propiedades y el número de qubits.
    """
    firstFoMResult = figureOfMeritResults[0]
    print("\n--- Checks FAILED ---")

    # Accede al diccionario de propiedades de forma segura
    properties = firstFoMResult.get('properties', {})

    # 1. Imprime el diccionario 'properties' completo
    print("\n--- Calculated Properties ---")
    pretty_properties = json.dumps(properties, indent=2)
    print(pretty_properties)


    return None

# Duda:
Cual es la manera de definirlo, es necesaria la decision function?

In [ ]:
from qonscious.checks import MeritComplianceCheck
from qonscious.foms import AggregateQPUFigureOfMerit

#check_QPU = MeritComplianceCheck(
#   figure_of_merit=AggregateQPUFigureOfMerit()
    #non decision function, question: if always true, why on_fail is mandatory?
#)
def requires_qbits_over(threshold: int):
    return lambda r: r["properties"]["n_qubits"] > threshold

# New check with decision function that requires more than n qubits, modify as needed
check_QPU= MeritComplianceCheck(
    figure_of_merit=AggregateQPUFigureOfMerit(),
    decision_function=requires_qbits_over(4)
)



In [ ]:

import os

from qonscious import run_conditionally
from qonscious.actions import QonsciousCallable
from qonscious.adapters import IBMSamplerAdapter


# we dont run any circuit, just the checks
qonscious_result = run_conditionally(
    backend_adapter=adapter,
    checks=[check_QPU],
    on_pass=QonsciousCallable(on_pass),
    on_fail=QonsciousCallable(on_fail)
)


You can create an adapter on IBM's least bussy (real) quantum computer as follows:

In [ ]:
import os
from qonscious.adapters import IBMSamplerAdapter
ibm_token = os.getenv("IBM_QUANTUM_TOKEN") # edit as needed
adapter = IBMSamplerAdapter.least_busy_backend(ibm_token)

You can create an adapter that simulates one of IBM's real computers (using Aer's Simulator, and the real noise models)

In [ ]:
import os
from qonscious.adapters import AerSimulatorAdapter
ibm_token = os.getenv("IBM_QUANTUM_TOKEN") # edit as needed
adapter = AerSimulatorAdapter.based_on(ibm_token, backend_name="ibm_brisbane") # you can choose a different one

You can even create an adapter on IonQ's QPU. This example shows how to work with the simulator of Aria-1 (including its noise model).

In [ ]:
import os
from qonscious.adapters import IonQBackendAdapter
api_key = os.getenv("IONQ_API_KEY")
adapter = IonQBackendAdapter.aria_simulator(api_key)